# Add peer-based features using learned embeddings

This notebook will:

1. Load a dataset and train a neural network to learn embeddings for categorical features.
2. Extract the embeddings
3. Construct peer-based features using the learned embeddings.
4. Export the new features for use in other models.

In [ ]:
# python packages
import json
import pandas as pd
from pathlib import Path

# modules we wrote
from src.ml_backend import load_processed  # load data
from src.ml_backend import NeuralNetworkWithEmbeddings  # model
from src.ml_backend import split_data  # data splitting

## Preparation

- load preprocessed data
- load data split configs

In [2]:
# Load configurations for data splitting, this would be the input for the `split_data` function
with open(Path("configs.json"), "r") as f:
    configs = json.load(f)["train_iteration"]

train_start: int = configs["train_start"]
val_start: int = configs["val_start"]
val_years: int = configs["val_years"]
test_years: int = configs["test_years"]
test_end: int = configs["test_end"]

# load processed data
preprocessed_data = load_processed(reprocesse=False)  # change to True if reprocessing is needed
    

## Train neural network with embeddings

The models will be defined in `src/ml_backend/models.py` as `NeuralNetworkWithEmbeddings`.

All models defined in this project have a consistent interface for training and prediction.

- `model.auto_tune()` will auto-tune hyperparameters (defined in `configs.json`
- `model.train_final()` will train the model on training data + validation data
- `model.predict()` will generate predictions on test data

In [ ]:
# loop for training iterations
predictions_list = []
while True:
    train_df, val_df, test_df = split_data(
        df = preprocessed_data,
        train_start=train_start,
        val_start=val_start,
        val_years=val_years,
        test_years=test_years
    )
    
    model = NeuralNetworkWithEmbeddings(
        train_df=train_df,
        val_df=val_df,
        test_df=test_df
    )

    model.auto_tune()
    model.train_final()
    predictions = model.predict()




    predictions_list.append(predictions)
    # Construct peer-based features for train + validation set (use learned embeddings)
    try:
        new_features_df = model.distance_weighted_feature()
        out_path = Path(f"data/train/peer_based_features_{val_start}_{val_start + val_years - 1}.csv")
        out_path.parent.mkdir(parents=True, exist_ok=True)
        # save with index (which should contain date/permno if present)
        new_features_df.to_csv(out_path, index=True)
    except Exception as exc:
        print(f"Failed to compute peer features for train+val: {exc}")

    print(f"Completed predictions for test period {val_start +val_years} to {val_start + val_years + test_years - 1}")
    # Update for next iteration
    val_start += test_years
    if val_start + val_years + test_years - 1 > test_end:
        break

# Combine all predictions
all_predictions = pd.concat(predictions_list, ignore_index=True)
all_predictions.to_csv(Path("data/test/predictions_with_embeddings.csv"), index=False)

==================== New Iteration ====================
start auto-tuning for test period 2009 to 2009


[I 2025-11-26 12:25:16,991] A new study created in memory with name: no-name-28c2d89c-b600-4f59-8f37-515d6171e6c1
/opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-11-26 12:25:16.995806: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2025-11-26 12:25:16.995841: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-11-26 12:25:16.995849: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2025-11-26 12:25:16.995865: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-26 12:25:16.995878: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:loca

NeuralNetworkWithEmbeddings tuning complete. Best val_loss=0.032604, params={'hidden_layers': 2, 'neurons_per_layer': 32, 'activation_functions': 'relu', 'learning_rates': 0.01, 'batch_sizes': 2048, 'embedding_dims': 8}
start final training for test period 2009 to 2009


/opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


NeuralNetworkWithEmbeddings final training complete. Params={'hidden_layers': 2, 'neurons_per_layer': 32, 'activation_functions': 'relu', 'learning_rates': 0.01, 'batch_sizes': 2048, 'embedding_dims': 8}
start prediction for test period 2009 to 2009
2244/2244 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step
Completed predictions for test period 2009 to 2009
==================== New Iteration ====================
start auto-tuning for test period 2010 to 2010


...

NeuralNetworkWithEmbeddings final training complete. Params={'hidden_layers': 3, 'neurons_per_layer': 32, 'activation_functions': 'relu', 'learning_rates': 0.01, 'batch_sizes': 2048, 'embedding_dims': 8}
start prediction for test period 2019 to 2019
2087/2087 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step
2087/2087 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step
Completed predictions for test period 2019 to 2019
Completed predictions for test period 2019 to 2019
==================== New Iteration ====================
start auto-tuning for test period 2020 to 2020
==================== New Iteration ====================
start auto-tuning for test period 2020 to 2020


[I 2025-11-26 14:39:37,237] A new study created in memory with name: no-name-cfd383b5-10c3-4b5e-bf07-1d155cb46c33
/opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
[I 2025-11-26 14:40:45,457] Trial 0 finished with value: 13.1035795211792 and parameters: {'hidden_layers': 4, 'neurons_per_layer': 32, 'activation_functions': 'relu', 'learning_rates': 0.01, 'batch_sizes': 2048, 'embedding_dims': 8}. Best is trial 0 with value: 13.1035795211792.
[I 2025-11-26 14:40:45,457] Trial 0 finished with value: 13.1035795211792 and parameters: {'hidden_layers': 4, 'neurons_per_layer': 32, 'activation_functions': 'relu', 'learning_rates': 0.01, 'batch_sizes': 

NeuralNetworkWithEmbeddings tuning complete. Best val_loss=0.031345, params={'hidden_layers': 2, 'neurons_per_layer': 32, 'activation_functions': 'relu', 'learning_rates': 0.01, 'batch_sizes': 2048, 'embedding_dims': 8}
start final training for test period 2020 to 2020


/opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


NeuralNetworkWithEmbeddings final training complete. Params={'hidden_layers': 2, 'neurons_per_layer': 32, 'activation_functions': 'relu', 'learning_rates': 0.01, 'batch_sizes': 2048, 'embedding_dims': 8}
start prediction for test period 2020 to 2020
2095/2095 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step
2095/2095 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step
Completed predictions for test period 2020 to 2020
Completed predictions for test period 2020 to 2020
